# LSTM 모델
LSTM(Long Short Term Memory networks)은 RNN의 한 종류로, 이전 단계의 정보를 memory cell에 저장한 후 전달한다. 현재 시점의 정보를 바탕으로 과거의 정보를 얼마나 잊을지 곱해주고, 그 결과에 현재의 정보를 더해 다음 시점으로 정보를 전달한다.

1. LSTM 모델 사용 이유
  - 과거의 정보를 기억하는데 뛰어나기 때문에 시간의 흐름(전반경기, 후반경기)이 존재하는 야구 데이터에 알맞을 것이라 추측하였다.  
 
  
2. 모델 결과 요약  
 - 다른 모델에 비해 높은 MSE값이 나와 성능이 좋지 않다고 판단된다.  
 
||MSE|  
|-|-|  
|PCT|0.01508519570698445|
|AVG|0.0003324946113552692|
|ERA|1.0642986325243158|

3. 모델 한계
  - EDA를 통해 시즌 간의 관계가 없음을 파악하였기 때문에 시즌/구단 각각에 대해 모델을 구축했다. 이로인해 augmentation을 했음에도 사용할 수 있는 데이터의 개수가 감소하여(약 3000 -> 약 70) 학습된 모델이 일반화가 되기 어려워 outlier에 취약해지는 한계점이 나타났다.

## 1. 패키지 로드
모델에 사용될 패키지와 모듈을 로드한다.

In [31]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from math import sqrt

## 2. 모델 정의

- input_shape: (timestep, feature)
- loss: mean_squared_error
- optimizer: Adam, lr=0.01
- Earlystopping: loss값의 min기준, patience = 2
- batch_size: 1

LSTM 모델에 timestep은 2를 가진 데이터를 입력값으로 사용하였습니다. Optimizer는 Adam방식을 사용하였고, learning rate를 0.001~0.1까지 조정해보며 성능을 판단한 결과 최적의 값으로 0.01을 설정하게 되었다. 모델의 오버피팅을 방지하기 위해 loss값을 2번 연속으로 상승한다면(min의 반대) 학습을 종료시키도록 설정했다. 데이터의 개수가 작기 때문에 batch_size를 1로 두고 시즌/구단별 모델을 각각 구축해 학습시킨 후 실제값과 예측값의 MSE를 기준으로 성능을 평가하였다.

In [37]:
def LSTM_model(X, y):
    optimizer = Adam(lr=0.01)
    
    model = Sequential()
    model.add(LSTM(100, input_shape=(X.shape[1],X.shape[2]))) # (timestep, feature)
    model.add(Dense(1)) # output = 1
    model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

    model.summary()

    early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

    hist = model.fit(X, y, epochs=100,
              batch_size=1, verbose=1, callbacks=[early_stop])
    
    return model, hist

In [38]:
def model_training(train_X, train_y, test_X, test_y, year, team, target):
    model_dict = dict()
    hist_dict = dict()
    test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID',target,'y_pred'])

    idx = 0
    for y in year:
        tmp1 = train_X[train_X["YEAR"] == y]
        tmp2 = train_y[train_y["YEAR"] == y]
        tmp3 = test_X[test_X["YEAR"] == y]
        tmp4 = test_y[test_y["YEAR"] == y]
        for t in team:
            name = '{}{}'.format(y,t)
            print(name,"=======================================")

            X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
            y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
            X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
            y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)

            X_train_v = X_train.values
            y_train_v = y_train.values

            X_test_v = X_test.values
            y_test_v = y_test.values

            X_train_t = X_train_v.reshape(X_train_v.shape[0], 2,X_train_v.shape[1]//2)
            X_test_t = X_test_v.reshape(X_test_v.shape[0], 2,X_test_v.shape[1]//2)

            model_dict[name], hist_dict[name] = LSTM_model(X_train_t, y_train_v)

            y_pred = model_dict[name].predict(X_test_t)

            test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0]]

            idx += 1

    test_pred_df[[target,'y_pred']] = test_pred_df[[target,'y_pred']].astype(float)
    
    return model_dict, hist_dict, test_pred_df

## 3. 승률/타율/방어율 예측

LSTM모델의 입력값을 timestep = 2로 설정했기 때문에 추가로 전처리한 데이터를 입력값으로 사용하였다. 또한, 누적 데이터보다 누적을 하지 않은 데이터를 사용했을 때 성능이 더 좋았다. 따라서 누적하지 않은 데이터를 24g경기씩 묶은 후에 augmentation시켜 사용했다.
 
##### 예시 데이터 구성) 2016, 한화(HH)
* timestep을 2로 설정하여 shift된 데이터 2묶음으로 구성됨

|shift_AVG_1|shift_PCT_1|...|shift_AVG_2|shift_PCT_2|...|
|-|-|-|-|-|-|
|0.3|0.5| ... |0.28|0.38|...|

* 이후 모델에 넣기 전 reshape를 이용해 차원을 조정한 후 입력값으로 사용함  
 현재 shape: (data_size, feature_size) -> 변경 shape: (data_size, 2, feature_size//2)

### 4.1. 승률 

In [15]:
PCT_lstm_train_X = pd.read_csv("lstmPCT/PCT_lstm_final_train_X.csv")
PCT_lstm_train_y = pd.read_csv("lstmPCT/PCT_lstm_final_train_y.csv")

PCT_lstm_test_X = pd.read_csv("lstmPCT/PCT_lstm_final_test_X.csv")
PCT_lstm_test_y = pd.read_csv("lstmPCT/PCT_lstm_final_test_y.csv")

In [16]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [39]:
pct_model, pct_hist, pct_test_df = model_training(PCT_lstm_train_X, PCT_lstm_train_y, PCT_lstm_test_X, 
                                                  PCT_lstm_test_y, year, team, 'PCT')

2016LG =======================================
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100)               63200     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 3s 59ms/sample - loss: 0.3428 - mae: 0.3251
Epoch 2/100
49/49 [==============================] - 0s 6ms/sample - loss: 0.0268 - mae: 0.1496
Epoch 3/100
49/49 [==============================] - 0s 6ms/sample - loss: 0.0271 - mae: 0.1423
Epoch 4/100
49/49 [==============================] - 0s 6ms/sample - loss: 0.0228 - mae: 0.1424
Epoch 5/100
49/49 [==============================] - 0s 6ms

49/49 [==============================] - 3s 69ms/sample - loss: 0.4186 - mae: 0.2786
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0028 - mae: 0.0426
Epoch 3/100
49/49 [==============================] - 0s 7ms/sample - loss: 0.0025 - mae: 0.0399
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0026 - mae: 0.0390
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0025 - mae: 0.0383
Epoch 00005: early stopping
2016WO =======================================
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 100)               63200     
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________

49/49 [==============================] - 3s 55ms/sample - loss: 0.2699 - mae: 0.2605
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0084 - mae: 0.0826
Epoch 3/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0090 - mae: 0.0801
Epoch 4/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0088 - mae: 0.0799
Epoch 00004: early stopping
2017NC =======================================
Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 100)               63200     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [========================

49/49 [==============================] - 3s 60ms/sample - loss: 0.2004 - mae: 0.2559
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0086 - mae: 0.0766
Epoch 3/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0082 - mae: 0.0764
Epoch 4/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0071 - mae: 0.0666
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0117 - mae: 0.0894
Epoch 6/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0072 - mae: 0.0665
Epoch 00006: early stopping
2017SS =======================================
Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 100)               63200     
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 101       
Total p

49/49 [==============================] - 3s 59ms/sample - loss: 0.3578 - mae: 0.2954
Epoch 2/100
49/49 [==============================] - 0s 10ms/sample - loss: 0.0078 - mae: 0.0745
Epoch 3/100
49/49 [==============================] - 1s 11ms/sample - loss: 0.0067 - mae: 0.0695
Epoch 4/100
49/49 [==============================] - 0s 7ms/sample - loss: 0.0080 - mae: 0.0750
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0051 - mae: 0.0574
Epoch 6/100
49/49 [==============================] - 0s 9ms/sample - loss: 0.0067 - mae: 0.0630
Epoch 7/100
49/49 [==============================] - 0s 9ms/sample - loss: 0.0037 - mae: 0.0481
Epoch 8/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0040 - mae: 0.0526
Epoch 9/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0046 - mae: 0.0545
Epoch 00009: early stopping
2018HT =======================================
Model: "sequential_39"
_______________________________________________

49/49 [==============================] - 3s 59ms/sample - loss: 0.2393 - mae: 0.2374
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0025 - mae: 0.0394
Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0026 - mae: 0.0418
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0033 - mae: 0.0475
Epoch 00004: early stopping
2019KT =======================================
Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 100)               63200     
_________________________________________________________________
dense_44 (Dense)             (None, 1)                 101       
Total params: 63,301
Trainable params: 63,301
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [========================

In [45]:
pct_test_df

,YEAR,T_ID,PCT,y_pred
0,2016,LG,0.608696,0.511131
1,2016,HH,0.500000,0.511373
2,2016,NC,0.565217,0.523035
3,2016,HT,0.458333,0.562959
4,2016,SK,0.458333,0.451833
5,2016,KT,0.291667,0.310401
6,2016,WO,0.375000,0.597372
7,2016,LT,0.500000,0.309653
8,2016,SS,0.500000,0.421670
9,2016,OB,0.666667,0.566030


In [49]:
pct_mse = mean_squared_error(pct_test_df['PCT'], pct_test_df['y_pred'])
pct_mse

0.01508519570698445

### 4.2. 타율

In [40]:
AVG_lstm_train_X = pd.read_csv("lstmAVG/AVG_lstm_final_train_X.csv")
AVG_lstm_train_y = pd.read_csv("lstmAVG/AVG_lstm_final_train_y.csv")
AVG_lstm_test_X = pd.read_csv("lstmAVG/AVG_lstm_final_test_X.csv")
AVG_lstm_test_y = pd.read_csv("lstmAVG/AVG_lstm_final_test_y.csv")

In [44]:
avg_model, avg_hist, avg_test_df = model_training(AVG_lstm_train_X, AVG_lstm_train_y, AVG_lstm_test_X, 
                                                  AVG_lstm_test_y, year, team, 'AVG')

2016LG =======================================
Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_49 (LSTM)               (None, 100)               58800     
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 101       
Total params: 58,901
Trainable params: 58,901
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 3s 57ms/sample - loss: 0.2555 - mae: 0.2325
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.7103e-04 - mae: 0.0096
Epoch 3/100
49/49 [==============================] - 0s 4ms/sample - loss: 5.1094e-05 - mae: 0.0062
Epoch 4/100
49/49 [==============================] - 0s 4ms/sample - loss: 5.7018e-05 - mae: 0.0062
Epoch 5/100
49/49 [============================

49/49 [==============================] - 0s 5ms/sample - loss: 3.0537e-05 - mae: 0.0044
Epoch 15/100
49/49 [==============================] - 0s 5ms/sample - loss: 3.4055e-05 - mae: 0.0047
Epoch 16/100
49/49 [==============================] - 0s 4ms/sample - loss: 2.7361e-05 - mae: 0.0042
Epoch 17/100
49/49 [==============================] - 0s 4ms/sample - loss: 2.5371e-05 - mae: 0.0040
Epoch 18/100
49/49 [==============================] - 0s 4ms/sample - loss: 3.2253e-05 - mae: 0.0047
Epoch 19/100
49/49 [==============================] - 0s 4ms/sample - loss: 2.2797e-05 - mae: 0.0039
Epoch 20/100
49/49 [==============================] - 0s 4ms/sample - loss: 2.5609e-05 - mae: 0.0038
Epoch 21/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.7445e-05 - mae: 0.0041
Epoch 00021: early stopping
2017LG =======================================
Model: "sequential_66"
_________________________________________________________________
Layer (type)                 Output Shape

49/49 [==============================] - 3s 57ms/sample - loss: 0.1882 - mae: 0.2111
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.4366e-04 - mae: 0.0130
Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.2540e-04 - mae: 0.0096
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.1287e-04 - mae: 0.0091
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.0664e-04 - mae: 0.0083
Epoch 6/100
49/49 [==============================] - 0s 4ms/sample - loss: 1.1681e-04 - mae: 0.0087
Epoch 7/100
49/49 [==============================] - 0s 4ms/sample - loss: 1.0578e-04 - mae: 0.0083
Epoch 8/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.0834e-04 - mae: 0.0087
Epoch 9/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.2424e-04 - mae: 0.0090
Epoch 00009: early stopping
2017KT =======================================
Model: "sequential_71"
_________________

49/49 [==============================] - 3s 58ms/sample - loss: 0.2111 - mae: 0.2101
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.2715e-04 - mae: 0.0125
Epoch 3/100
49/49 [==============================] - 0s 4ms/sample - loss: 2.7264e-04 - mae: 0.0133
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.3574e-04 - mae: 0.0127
Epoch 00004: early stopping
2018SK =======================================
Model: "sequential_80"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_73 (LSTM)               (None, 100)               58800     
_________________________________________________________________
dense_73 (Dense)             (None, 1)                 101       
Total params: 58,901
Trainable params: 58,901
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [============

49/49 [==============================] - 0s 4ms/sample - loss: 1.2743e-04 - mae: 0.0084
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.1185e-04 - mae: 0.0088
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 8.4051e-05 - mae: 0.0071
Epoch 6/100
49/49 [==============================] - 0s 4ms/sample - loss: 7.2073e-05 - mae: 0.0067
Epoch 7/100
49/49 [==============================] - 0s 4ms/sample - loss: 7.6004e-05 - mae: 0.0073
Epoch 8/100
49/49 [==============================] - 0s 5ms/sample - loss: 4.7426e-05 - mae: 0.0056
Epoch 9/100
49/49 [==============================] - 0s 5ms/sample - loss: 5.5799e-05 - mae: 0.0060
Epoch 10/100
49/49 [==============================] - 0s 5ms/sample - loss: 3.4722e-05 - mae: 0.0049
Epoch 11/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.8522e-05 - mae: 0.0041
Epoch 12/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.8051e-05 - mae: 0.0042
Epoch 13/

Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.9072e-04 - mae: 0.0131
Epoch 4/100
49/49 [==============================] - 0s 6ms/sample - loss: 2.4716e-04 - mae: 0.0127
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.5248e-04 - mae: 0.0121
Epoch 6/100
49/49 [==============================] - 0s 4ms/sample - loss: 2.3452e-04 - mae: 0.0119
Epoch 7/100
49/49 [==============================] - 0s 6ms/sample - loss: 2.0260e-04 - mae: 0.0123
Epoch 8/100
49/49 [==============================] - 0s 5ms/sample - loss: 2.3185e-04 - mae: 0.0125
Epoch 9/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.1037e-04 - mae: 0.0084
Epoch 10/100
49/49 [==============================] - 0s 6ms/sample - loss: 1.1090e-04 - mae: 0.0088
Epoch 11/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.2640e-04 - mae: 0.0092
Epoch 00011: early stopping
2019SS =======================================
Model: "sequential_94"


In [46]:
avg_test_df

,YEAR,T_ID,AVG,y_pred
0,2016,LG,0.296069,0.289743
1,2016,HH,0.288575,0.305112
2,2016,NC,0.287440,0.302271
3,2016,HT,0.256739,0.300588
4,2016,SK,0.305263,0.305513
5,2016,KT,0.295455,0.281148
6,2016,WO,0.289941,0.307299
7,2016,LT,0.309893,0.281632
8,2016,SS,0.283863,0.272717
9,2016,OB,0.298225,0.298349


In [48]:
avg_mse = mean_squared_error(avg_test_df['AVG'], avg_test_df['y_pred'])
avg_mse

0.0003324946113552692

### 4.3. 방어율

In [50]:
ERA_lstm_train_X = pd.read_csv("lstmERA/ERA_lstm_final_train_X.csv")
ERA_lstm_train_y = pd.read_csv("lstmERA/ERA_lstm_final_train_y.csv")
ERA_lstm_test_X = pd.read_csv("lstmERA/ERA_lstm_final_test_X.csv")
ERA_lstm_test_y = pd.read_csv("lstmERA/ERA_lstm_final_test_y.csv")

In [51]:
era_model, era_hist, era_test_df = model_training(ERA_lstm_train_X, ERA_lstm_train_y, ERA_lstm_test_X, 
                                                  ERA_lstm_test_y, year, team, 'ERA')

2016LG =======================================
Model: "sequential_96"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_89 (LSTM)               (None, 100)               58400     
_________________________________________________________________
dense_89 (Dense)             (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 3s 59ms/sample - loss: 1.5338 - mae: 0.8908
Epoch 2/100
49/49 [==============================] - 0s 8ms/sample - loss: 0.6699 - mae: 0.7295
Epoch 3/100
49/49 [==============================] - 0s 9ms/sample - loss: 0.5252 - mae: 0.6265
Epoch 4/100
49/49 [==============================] - 0s 9ms/sample - loss: 0.4371 - mae: 0.5337
Epoch 5/100
49/49 [==============================] - 0s 7ms

49/49 [==============================] - 0s 5ms/sample - loss: 0.7838 - mae: 0.6893
Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.7831 - mae: 0.6699
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.8239 - mae: 0.7203
Epoch 5/100
49/49 [==============================] - 0s 4ms/sample - loss: 1.0717 - mae: 0.8597
Epoch 00005: early stopping
2017NC =======================================
Model: "sequential_108"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_101 (LSTM)              (None, 100)               58400     
_________________________________________________________________
dense_101 (Dense)            (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [========================

49/49 [==============================] - 3s 57ms/sample - loss: 1.5787 - mae: 1.0213
Epoch 2/100
49/49 [==============================] - 0s 4ms/sample - loss: 1.1934 - mae: 0.8165
Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.1130 - mae: 0.7882
Epoch 4/100
49/49 [==============================] - 0s 6ms/sample - loss: 1.3120 - mae: 0.9454
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 1.2188 - mae: 0.8578
Epoch 00005: early stopping
2017SS =======================================
Model: "sequential_114"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_107 (LSTM)              (None, 100)               58400     
_________________________________________________________________
dense_107 (Dense)            (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
________________________________

49/49 [==============================] - 3s 62ms/sample - loss: 1.2225 - mae: 0.8966
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.7674 - mae: 0.7415
Epoch 3/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.6935 - mae: 0.6905
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.7713 - mae: 0.7599
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.8396 - mae: 0.7455
Epoch 00005: early stopping
2018KT =======================================
Model: "sequential_121"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_114 (LSTM)              (None, 100)               58400     
_________________________________________________________________
dense_114 (Dense)            (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
________________________________

49/49 [==============================] - 3s 58ms/sample - loss: 1.0313 - mae: 0.6713
Epoch 2/100
49/49 [==============================] - 0s 6ms/sample - loss: 0.5150 - mae: 0.5798
Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.4891 - mae: 0.5422
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.5003 - mae: 0.5463
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.4866 - mae: 0.5310
Epoch 6/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.6813 - mae: 0.6501
Epoch 7/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.5536 - mae: 0.6232
Epoch 00007: early stopping
2019NC =======================================
Model: "sequential_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_121 (LSTM)              (None, 100)               58400     
__________________________________________

49/49 [==============================] - 3s 61ms/sample - loss: 0.9113 - mae: 0.6202
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.4381 - mae: 0.5469
Epoch 3/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.3568 - mae: 0.4848
Epoch 4/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.3245 - mae: 0.4698
Epoch 5/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.3321 - mae: 0.4821
Epoch 6/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.3304 - mae: 0.4819
Epoch 00006: early stopping
2019SS =======================================
Model: "sequential_134"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_127 (LSTM)              (None, 100)               58400     
_________________________________________________________________
dense_127 (Dense)            (None, 1)                 101       
Total 

In [52]:
era_test_df

,YEAR,T_ID,ERA,y_pred
0,2016,LG,3.649770,5.578984
1,2016,HH,5.258114,6.274403
2,2016,NC,3.575342,5.376220
3,2016,HT,4.120827,5.723327
4,2016,SK,5.144882,5.243700
5,2016,KT,6.314516,7.289543
6,2016,WO,5.704839,5.273979
7,2016,LT,5.849294,5.683576
8,2016,SS,5.400000,6.223722
9,2016,OB,4.710280,5.277182


In [53]:
era_mse = mean_squared_error(era_test_df['ERA'], era_test_df['y_pred'])
era_mse

1.0642986325243158